# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [67]:
import pandas as pd
import numpy as np
from scipy import stats

In [68]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [69]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [70]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [71]:
n = sum(data.race=='w')
p = sum(data[data.race=='w'].call) / n
print(n*p, n*(1-p))
n = sum(data.race=='b')
p = sum(data[data.race=='b'].call) / n
print(n*p, n*(1-p))

235.0 2200.0
157.0 2278.0


1. The appropriate test of this problem is a two-sample hypothesis test on population proportions. The central limit theorem applies as both np and n(1-p) for both populations is >= 10

2. The null hypothesis is that there is no difference between the number of call backs for black and white sounding names.

The alternative hypothesis is that there is a difference between the number of call backs for black and white sounding names.

In statistical terms:
let p1 = proportion of call backs for black sounding names and p2 = proportion of call backs for white sounding names

Null hypothesis: p1 - p2 = 0

Alt Hypothesis: p1 - p2 != 0

In [72]:
w = data[data.race=='w']
b = data[data.race=='b']

### Frequentist Approach

In [73]:
# Sample proportions, size, and differences
w_size = len(w)
w_p = np.sum(data[data.race=='w'].call) / w_size
b_size = len(b)
b_p = np.sum(data[data.race=='b'].call) / b_size
diff_p = b_p - w_p
pooled_p = (b_p + w_p) / (b_size + w_size)

#Margin of Error
SE_b = np.sqrt(b_p*(1-b_p)/b_size)
SE_w = np.sqrt(w_p *(1-w_p)/w_size)
SE_diff = np.sqrt(SE_b**2 + SE_w**2)

#95% CI for the difference in proportions
diff_conf_int_95 = [diff_p - 1.96*SE_diff, diff_p + 1.96*SE_diff]

#
z_score = (diff_p - 0) / (SE_diff)
p_val = 1 - stats.norm.cdf(np.abs(z_score))

In [74]:
print("Margin of Error: +-", SE_diff)
print("95% CI: ", diff_conf_int_95)
print("z-score: ", z_score)
print("p-value: ", p_val)

Margin of Error: +- 0.00778337058668
95% CI:  [-0.047288260559332024, -0.016777447859559147]
z-score:  -4.11555043573
p-value:  1.93128260376e-05


### Boostrap Approach

In [94]:
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data)))

    return bs_replicates
def get_prop(data):
    return np.sum(data) / len(data)

#Bootstrap replicates of blacks and whites
bs_rep_b = draw_bs_reps(b.call, get_prop, 10000)
bs_rep_w = draw_bs_reps(w.call, get_prop, 10000)
bs_rep_b_mean = np.mean(bs_rep_b)
bs_rep_b_std = np.std(bs_rep_b)
bs_rep_w_mean = np.mean(bs_rep_w)
bs_rep_w_std = np.std(bs_rep_w)

#Bootstrap replicate differences
bs_replicates = bs_rep_b - bs_rep_w
bs_replicates_mean = np.mean(bs_replicates)
bs_replicates_std = np.std(bs_replicates)

bs_95_CI = np.percentile(bs_replicates, [2.5, 97.5])
p_val = np.sum(bs_replicates <= diff_p) / len(bs_replicates)

In [95]:
print("Margin of Error: +-", bs_replicates_std)
print("95% CI: ", bs_95_CI)
print("p-value: ", p_val)

Margin of Error: +- 0.00776185739214
95% CI:  [-0.0476386  -0.01683778]
p-value:  0.5067


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [77]:
bs_rep_b 

array([ 0.06365503,  0.06447639,  0.06570842, ...,  0.062423  ,
        0.06735113,  0.06160164])

In [78]:
print(SE_diff, bs_replicates_std)

0.00778337058668 0.00781692246327


In [83]:
type(w)

pandas.core.frame.DataFrame

In [86]:
combined_p

0.080492813141683772